In [52]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime as datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
def dataLoad():
    return pd.read_excel("./Location.xlsx").applymap(str.lower)

In [24]:
def cheapflightsearch(startlocation ,endlocation , startdate , enddate,dataframe):
    airline = "";
    price = "";
    if(not((isValidLocation(dataframe,startlocation)) & (isValidLocation(dataframe,endlocation)))):
        return ("Invalid Location" , "-1")
    elif(not ((isValidDateFormat(startdate)) & ((isValidDateFormat(enddate) | (enddate is None) | (enddate ==""))))):
        return ("Invalid Date" , "-1")
    return (airline , price)

In [4]:
def isValidLocation(dataframe , location):
    return (location.lower() in dataframe['Name'].unique()) | (location.lower() in dataframe['Code'].unique())

In [5]:
def isValidDateFormat(dateValue):
    format = "%Y-%m-%d"
    if dateValue is None:
        return False
    else:
        try:
            datetime.datetime.strptime(dateValue, format)
            return True
        except ValueError:
            return False

In [71]:
def callCheapFlightStatus(source,destination,startDate,endDate):
    dataframe = dataLoad()
    result = cheapflightsearch(source,destination,startDate,endDate,dataframe)
    if (result[0]==""):
        driver = webdriver.Chrome('./chromedriver')
        driver.get("https://www.makemytrip.com/")
        try:
            driver.maximize_window()
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html')))
            driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/label/input").click()
            time.sleep(3)
            sourceFiled= driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/div/div/div/input")
            sourceFiled.clear()
            sourceFiled.send_keys(source)
            time.sleep(3)
            driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/div/div/div/div/div/ul/li[1]/div/div[1]").click()
            time.sleep(3)
            destinationPath=driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/input")
            destinationPath.clear()
            destinationPath.send_keys(destination)
            time.sleep(5)
        except e:
            print("Error - " +e)
        finally:
            driver.close()
    else:
        print(result)

In [73]:
#callCheapFlightStatus("pnq","del","2022-06-02","")